In [18]:
import requests
import json
from PIL import Image as Img
from io import BytesIO
import boto3
from boto.s3.key import Key

url = 'http://ec2-34-207-73-202.compute-1.amazonaws.com/api/allImages'
resp = requests.get(url=url)
data = json.loads(resp.text)

In [19]:
data = json.loads(resp.text)

In [20]:
data

[{'Classification_Ccore_Id': 1,
  'Image_Id': 'https://info-arch-hate-images-corp.s3.amazonaws.com/http%3A--1.bp.blogspot.com-_9Bx0L3n3uAo-RvqvzrO-0qI-AAAAAAAAAsY-QFzSrXhgwz8-s400-coronado.jpg',
  'Score': 1,
  'Username': 'benny'},
 {'Classification_Ccore_Id': 2,
  'Image_Id': 'https://info-arch-hate-images-corp.s3.amazonaws.com/http%3A--1.bp.blogspot.com-_dLqkrp53BGg-THj_2DxqnMI-AAAAAAAABng-v5NlOWVdjyQ-s1600-Swastika%2BBuddhist.jpg',
  'Score': 0,
  'Username': 'benny'},
 {'Classification_Ccore_Id': 3,
  'Image_Id': 'https://info-arch-hate-images-corp.s3.amazonaws.com/http%3A--1.bp.blogspot.com-_FvLoFOjAIpg-S7CW1DsC0WI-AAAAAAAAExw-_wlhvKnScUI-s1600-Most%2BHot%2BBengali%2BActress%2BSwastika%2BHot%2BPhoto%2BGallery%2B%286%29.bmp',
  'Score': 1,
  'Username': 'benny'},
 {'Classification_Ccore_Id': 4,
  'Image_Id': 'https://info-arch-hate-images-corp.s3.amazonaws.com/http%3A--1.bp.blogspot.com-_FvLoFOjAIpg-TGfZh5ZICiI-AAAAAAAAFWI-aBX6uU3zzc8-s1600-Most%2BHot%2BBengali%2BActress%2BSwastik

In [7]:
data[4]['Score'] == 0

False

In [8]:
import numpy as np

In [11]:
imageList = []
labels = []
unhandledCount = 0

for i in range(0,300):
    s3Link = data[i]['Image_Id']
    try:
        response = requests.get(s3Link)
        image = Img.open(BytesIO(response.content))
    except:
        unhandledCount = unhandledCount +1
        continue
    imageList.append(np.asarray(image.convert('RGB').resize((64,64))))
    labels.append(data[i]['Score'])
    #s3 = boto3.client('s3')
    #s3.put_object(Bucket='info-arch-hate-images-train-stage',Key=key,Body=response.content)
    

In [10]:
unhandledCount

11

In [14]:
imageList[0].shape

(64, 64, 3)

In [15]:
imageArray = np.array(imageList)

In [16]:
imageArray.shape

(295, 64, 64, 3)

In [17]:
tImageList = []
tImage = imageList[0]
tImageList.append(tImage)
np.array(tImageList).shape

(1, 64, 64, 3)

In [21]:
from sklearn.model_selection import train_test_split

(X_train, X_test, Y_train, Y_test) = train_test_split(imageArray, labels, test_size=0.25)

In [32]:
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten,Dense, Dropout
from keras.models import Sequential

In [33]:
input_shape = (64,64,3)
model = Sequential() 
model.add(Conv2D(32, (2, 2), input_shape=input_shape)) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
  
model.add(Conv2D(32, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
  
model.add(Conv2D(64, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
  
model.add(Flatten()) 
model.add(Dense(64)) 
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(1)) 
model.add(Activation('sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [34]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['acc'])

In [47]:
history = model.fit(X_train, Y_train, epochs = 1)

Epoch 1/1
221/221 [==============================] - 1s 5ms/step - loss: 8.2477 - acc: 0.4842


In [48]:
import sklearn.metrics as metrics

results_test = model.evaluate(X_test, Y_test)
y_preds = model.predict_classes(X_test)
print(metrics.classification_report(Y_test, y_preds))

74/74 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.53      1.00      0.69        39
           1       0.00      0.00      0.00        35

    accuracy                           0.53        74
   macro avg       0.26      0.50      0.35        74
weighted avg       0.28      0.53      0.36        74



In [45]:
len(y_preds[y_preds == 0])

74

In [46]:
len(y_preds[y_preds == 1])

0

In [216]:
name = 'primitive_model.h5'
model.save(name)

In [224]:
s3 = boto3.client('s3')
s3.put_object(Bucket='info-arch-hate-images-train-stage',Key=name,Body=open(name, 'rb'))


{'ResponseMetadata': {'RequestId': 'A7A4E05D5EFB9906',
  'HostId': 'LmHl9IlKWG/sPK2vieeUEFJ/Yi0bvkFoffjOAkO+XI4SAHP3HMhEesQyCaNdO7xPkzQvGp0a0e0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'LmHl9IlKWG/sPK2vieeUEFJ/Yi0bvkFoffjOAkO+XI4SAHP3HMhEesQyCaNdO7xPkzQvGp0a0e0=',
   'x-amz-request-id': 'A7A4E05D5EFB9906',
   'date': 'Sun, 16 Aug 2020 05:03:54 GMT',
   'etag': '"965074c74e4b96ffbb92b689df0c28ce"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"965074c74e4b96ffbb92b689df0c28ce"'}